In [ ]:
import pandas as pd
import numpy as np
import torch
import os
import glob
root = '/mnt/f/datasets/MSLS/train_val/'


In [ ]:
cities = os.listdir(root)
cities = [os.path.join(root, city) for city in cities]
os.listdir(cities[0])

In [72]:
def build_msls_city_metadata(city_path: str, temporal_radius: float =5.0, **kwargs):
    postprocessed = os.path.join(city_path, 'database', 'postprocessed.csv')
    seq_info = os.path.join(city_path, 'database', 'seq_info.csv')
    image_folder = os.path.join(city_path, 'images')
    df_postprocessed = pd.read_csv(postprocessed, index_col=0)
    df_seq_info = pd.read_csv(seq_info, index_col=0)

    spatial_groups = df_postprocessed.groupby('unique_cluster').groups
    temporal_groups = df_seq_info.groupby('sequence_key').groups

    paths = df_postprocessed['key'].apply(lambda x: os.path.join(image_folder, x + '.jpg')).tolist()
    sadj = df_postprocessed['unique_cluster'].apply(lambda x: spatial_groups[x].tolist()).tolist()
    tadj = df_seq_info['sequence_key'].apply(lambda x: temporal_groups[x])
    frame_number = df_seq_info['frame_number']

    for i, ta in tadj.items():
        filtered_ta = []
        for t in ta:
            if abs(frame_number[t] - frame_number[i]) <= temporal_radius:
                filtered_ta.append(t)
        tadj[i] = filtered_ta

    index = np.arange(len(paths))
    mapping = dict(zip(df_seq_info.index, index))
    tadj = tadj.apply(lambda x: [mapping[i] for i in x])
    tadj.tolist()
    data = pd.DataFrame({
        'id': index,
        'path': paths,
        'sadj': sadj,
        'tadj': tadj
    })
    return data

,id,path,sadj,tadj
22,0,/mnt/f/datasets/MSLS/train_val/amman/images/ot...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5]"
23,1,/mnt/f/datasets/MSLS/train_val/amman/images/B6...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6]"
24,2,/mnt/f/datasets/MSLS/train_val/amman/images/7h...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7]"
25,3,/mnt/f/datasets/MSLS/train_val/amman/images/8M...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8]"
26,4,/mnt/f/datasets/MSLS/train_val/amman/images/wl...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]"
...,...,...,...,...
1783,948,/mnt/f/datasets/MSLS/train_val/amman/images/8h...,"[947, 948, 949, 950, 951, 952]","[947, 948, 949, 950, 951, 952]"
1784,949,/mnt/f/datasets/MSLS/train_val/amman/images/7D...,"[947, 948, 949, 950, 951, 952]","[947, 948, 949, 950, 951, 952]"
1785,950,/mnt/f/datasets/MSLS/train_val/amman/images/ap...,"[947, 948, 949, 950, 951, 952]","[947, 948, 949, 950, 951, 952]"
1786,951,/mnt/f/datasets/MSLS/train_val/amman/images/gz...,"[947, 948, 949, 950, 951, 952]","[947, 948, 949, 950, 951, 952]"


In [73]:
from torch import nn
import torch

l1 = nn.Linear(512, 512)
l2 = nn.Linear(512, 512)
l3 = nn.Linear(512, 512)



In [79]:
x = torch.randn(1, 1, 512)
q = l1(x)
k = l2(x)
v = l3(x)
import torch.nn.functional as F

o1 = F.scaled_dot_product_attention(q, k, v)

In [80]:
y = torch.randn(1, 1, 512)
q = l1(y)
k = l2(y)
o2 = F.scaled_dot_product_attention(q, k, v)

In [81]:
o1 - o2

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [68]:
sadj

0      [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
1      [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
2      [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
3      [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
4      [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
                             ...                        
948                       [947, 948, 949, 950, 951, 952]
949                       [947, 948, 949, 950, 951, 952]
950                       [947, 948, 949, 950, 951, 952]
951                       [947, 948, 949, 950, 951, 952]
952                       [947, 948, 949, 950, 951, 952]
Name: unique_cluster, Length: 953, dtype: object

In [44]:
for i, ta in enumerate(tadj):
    filtered_ta = []
    for t in ta:
        print(i, t)
        if abs(frame_number[t] - frame_number[i]) <= temporal_radius:
            filtered_ta.append(t)
    tadj[i] = filtered_ta

0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 175
0 176
0 177
0 178
0 179
0 180
0 181
0 182
0 416
0 417
0 418
0 419
0 420
0 421
0 422
0 423
0 424
0 470
0 471
0 472
0 473
0 474
0 475
0 476
0 477
0 818
0 819
0 820
0 821
0 822
0 823
0 824
0 1377


IndexError: index 1377 is out of bounds for axis 0 with size 953

In [ ]:
tadj

In [ ]:
df_seq_info

In [ ]:
df_seq_info.groupby('sequence_key').groups

In [ ]:
images[0]

In [ ]:
groups = df.groupby('unique_cluster').groups


In [ ]:
cluster_keys

In [ ]:
os.listdir(database)

In [ ]:
for city in cities:
    database = os.path.join(city, 'database')
    query = os.path.join(city, 'query')
    